# Предсказание заработной платы по резюме

**Цель:** построить модель, которая может предсказывать зарплату по данным в резюме.

In [1]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import torch.nn as nn 
import transformers

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from category_encoders.count import CountEncoder 
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder

from tqdm.notebook import tqdm

**Описание названий столбцов** 
- Id - идентификацонный номер
- Title - название вакансии
- FullDescription - полное описание
- LocationRaw - местоположение
- LocationNormalized - нормализованное метсоположение
- ContractType - тип контракта
- ContractTime - время контракта (тип найма)
- Company - наименования компании
- Category - категория
- SalaryRaw - Заработная плата
- SalaryNormalized - Заработная плата нормализованная 
- SourceName - имя источника

In [2]:
RANDOM_STATE = 13

In [3]:
#менсяю тип занятости
def type_time(x):
    res = 'unknown'
    if x == 'full_time':
        res = 'permanent'
    elif x == 'part_time':
        res = 'contract'
    return res

In [4]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.precision', 3)
np.set_printoptions(precision=3, suppress=True)


In [5]:
path_colab = '/content/sample_data/train_rev1.csv'
path_local = 'train_rev1.csv'
path_kaggle = '/kaggle/input/df-salary/train_rev1.csv'

if os.path.exists(path_local):
    df = pd.read_csv(path_local, index_col = 'Id')
elif os.path.exists(path_colab):
    df = pd.read_csv(path_colab, index_col = 'Id')
elif os.path.exists(path_kaggle):
    df = pd.read_csv(path_kaggle, index_col = 'Id')
else:
    print('uncorrect path')


uncorrect path


In [ ]:
#проверим part_time
df[['ContractType', 'ContractTime']].query('ContractType == "part_time"').head()

In [ ]:
#общая инфо
df.info()

In [ ]:
#заполним пропуски в ContractTime
df.ContractTime = df.ContractTime.fillna(df.ContractType.transform(type_time))

In [ ]:
#удалим лишнии столбцы
data = df.drop(['LocationRaw', 'SalaryRaw', 'ContractType', 'SourceName'], axis=1)
data.head()

In [ ]:
#Количесвто уникальных значений Category, Company, LocationNormalized
len(data.Category.unique()), len(data.Company.unique()), len(data.LocationNormalized.unique())

In [ ]:
#удилил все пропуски
data = data.dropna()
data.info()

In [ ]:
#уменьшу выборку
df_bert = data.copy().sample(50000, random_state = RANDOM_STATE)
df_bert.shape

In [ ]:
#оставляем в тесктсе только буквы и в нижнем регистре
df_bert.FullDescription = df_bert.FullDescription.apply(
    lambda x: ' '.join(re.sub(r'[^a-zA-Z\']', ' ', x).split()).lower())
#проверим наличие пустых строк
display(df_bert.loc[df_bert.FullDescription == ''])

*Подготовка категориальных столбцов к дальнейше обработке*

In [ ]:
mask_cat = ['LocationNormalized', 'ContractTime', 'Company', 'Category']
df_cat = df_bert[mask_cat].reindex(columns=['ContractTime', 'LocationNormalized', 'Company', 'Category'])
df_cat.tail()

**BERT encoding**

In [ ]:
def get_vectors(model, tokenizer, data, device, max_length=None):
    batch_size = 100
    tokenized = tokenizer(data, 
                          padding='max_length',
                          add_special_tokens=True,
                          truncation=True,
                          max_length=max_length,
                          return_tensors="pt")
    
    mask = tokenized['attention_mask']
    input_ids = tokenized['input_ids']
    
    embeddings = []
    for i in tqdm(range(input_ids.shape[0] // batch_size)):
        batch = input_ids[batch_size * i : batch_size * (i + 1)].clone().detach().to(device)  
        attention_mask_batch = mask[batch_size * i : batch_size * (i + 1)].clone().detach().to(device) 
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].detach().cpu().numpy())
        
        del batch
        del attention_mask_batch
        del batch_embeddings
        
    return embeddings

In [ ]:
#получаем веса для берта
tokenizer = transformers.ConvBertTokenizer.from_pretrained('YituTech/conv-bert-base')
model = transformers.ConvBertModel.from_pretrained('YituTech/conv-bert-base')

In [ ]:
#настройка gpu или cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(f'Используется device - {device}')

In [ ]:
%%time
features_description = get_vectors(model, tokenizer, list(df_bert.FullDescription.values), device)

In [ ]:
%%time
features_title = get_vectors(model, tokenizer, list(df_bert.Title.values), device, 100)

In [ ]:
#соединяем фичи и забираем таргет
features = np.concatenate(features_description)
features = np.hstack([features, np.concatenate(features_title)])
features = np.hstack([features, df_cat.values])
target = df_bert.SalaryNormalized.values
features.shape

In [ ]:
#сохраняем фичи и таргет
with open('/kaggle/working/features', 'wb') as f, open('/kaggle/working/target', 'wb') as f2:
    np.save(f, features)
    np.save(f2, target)


In [ ]:
#Делим выборку
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    test_size=.1, 
    random_state=RANDOM_STATE)

In [ ]:
#энкодинг категориальных фичей
ce_ordinal_encoder = OrdinalEncoder()
ce_ordinal_encoder.fit(X_train[:, -4])
X_train_ordinal = ce_ordinal_encoder.transform(X_train[:, -4])
X_test_ordinal = ce_ordinal_encoder.transform(X_test[:, -4])

In [ ]:
#энкодинг категориальных фичей
ce_count_encoder = CountEncoder(normalize=True)
ce_count_encoder.fit(X_train[:, -3:])
X_train_encoder = ce_count_encoder.transform(X_train[:, -3:])
X_test_encoder = ce_count_encoder.transform(X_test[:, -3:])

In [ ]:
#приседееняем котегориальные фичи
X_train = np.hstack([X_train[:, :-4], X_train_ordinal])
X_test = np.hstack([X_test[:, :-4], X_test_ordinal])
X_train = np.hstack([X_train, X_train_encoder]).astype('float32')
X_test = np.hstack([X_test, X_test_encoder]).astype('float32')
X_train.shape

**Проверка на линейной моделе**

In [ ]:
model_lr_bert = LinearRegression().fit(X_train, y_train)
mean_squared_error(y_test, model_lr_bert.predict(X_test), squared=False)

**НЕЙРОСЕТЬ для предсказание з/п**

In [ ]:
class Regression_Net(torch.nn.Module):
    
    def __init__ (self, input_size, hidden_size, out_size):
        super(Regression_Net, self).__init__()
        self.drop_input = nn.Dropout(0.1)
        self.input = nn.BatchNorm1d(input_size, eps=1e-02)
        self.relu = nn.ReLU()
        self.hidden_dense = nn.Linear(input_size, hidden_size)
        self.relu_out = nn.ReLU()
        self.drop_out = nn.Dropout(0.1)
        self.output = nn.Linear(hidden_size, out_size)
       
    def forward(self, x):
        out = self.drop_input(self.input(x))
        hidden_dense = self.relu(self.hidden_dense(out))
        output = self.relu_out(self.drop_out(self.output(hidden_dense)))
        
        return output

In [ ]:
def train_model(model, device, train_loader, criterion, optimizer):
    model.train()
    y_true = []
    y_pred = []
    history= []
    
    for x_batch, y_batch in tqdm(train_loader):
        
        train_label = y_batch.to(device)
        input_id = x_batch.squeeze(1).to(device)
        
        model.zero_grad()
        output = model(input_id).squeeze(1)
        
        loss = criterion(output, train_label.to(torch.float32))
        history.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        #predictions
        pred = output.detach().cpu().numpy()        
        target = np.round(train_label.detach().cpu().numpy()) 
       
        y_pred.extend(pred.tolist())
        y_true.extend(target.tolist())
                              
    print(f'{i + 1}  Train loss: {history[-1]}')
    print("RMSE on training" , mean_squared_error(y_true,y_pred, squared=False))
    print('---------------------------------------------------------------------------------------')
    return history

In [ ]:
def test_model(model, device, test_loader):
    #тестирование
    prob = []
    y_pred = []
    y_true = []
    model.eval()
    
    with torch.no_grad():
        for x_batch, y_batch in tqdm(test_loader):

            test_label = y_batch.to(device)
            input_id = x_batch.squeeze(1).to(device)

            output = model(input_id).squeeze(1)
            #predictions
            pred = np.round(output.detach().cpu().numpy())
            target = np.round(test_label.detach().cpu().numpy()) 
            
            #prob = np.hstack([prob, output.detach().cpu().numpy()])          
            y_pred.extend(pred.tolist())
            y_true.extend(target.tolist())

    print("RMSE on test" , mean_squared_error(y_true, y_pred, squared=False))
    
    print('___________________________________________________________________________________________')

In [ ]:
#тензоры для трейн
labels = torch.tensor(y_train)
input_ids = torch.tensor(X_train)
input_ids.shape, labels.shape

In [ ]:
#тензоры для тест
labels_test = torch.tensor(y_test)
input_ids_test = torch.tensor(X_test)
input_ids_test.shape, labels_test.shape

In [ ]:
#создание дата лоадеров
train_dataset = TensorDataset(input_ids, labels)
test_dataset = TensorDataset(input_ids_test, labels_test)
train_dataloader = DataLoader(train_dataset, batch_size=100, drop_last=True) 
test_dataloader = DataLoader(test_dataset, batch_size=100, drop_last=True) 

In [ ]:
model = Regression_Net(X_train.shape[1], 150, 1)
model.to(device)
criterion = nn.MSELoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=0.008)
epoch = 50

In [ ]:
history_train = []
history_test = []
for i in range(epoch):
    history_train.extend(train_model(model, device, train_dataloader, criterion, optimizer))
    test_model(model, device, test_dataloader)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(history_train)
plt.title('Loss на каждой итерации батча')
plt.ylabel('RSMELoss')
plt.xlabel('batches')
plt.show()

**Вывод:**
В ходе исследования была использована и хорошо себя покзала предобученная нейронная сеть "BERT" "YituTech", использование эмбендингов которых даже на простой модели LinearRegression показало приемлемый результат RSME 13416, что не много хуже результата нейронной сети написанной с использованием библиотеки PyTorch - 13241.
Однако надо учесть  тот момент, что обучение некронки проходило не с оптимальными гипперпараметрами и всего на 50 эпохах.